In [1]:
#!/usr/bin/env python
import os
from PhysicsTools.NanoAODTools.postprocessing.framework.postprocessor import *
from PhysicsTools.NanoAODTools.postprocessing.trigger.triggerFilter import triggerFilter

from PhysicsTools.NanoAODTools.postprocessing.modules.common.countHistogramsModule import countHistogramsProducer
from PhysicsTools.NanoAODTools.postprocessing.framework.crabhelper import runsAndLumis


Welcome to JupyROOT 6.14/09


In [2]:
##
## python helper functions
##

# allows modules to be reloaded automatically without restarting kernel
%load_ext autoreload
%autoreload 2
# widens cells
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
# define some parameters for running code
isMC = True
base_path = '/eos/cms/store/group/phys_exotica/bffZprime/sample_files/'
infile = '{}BFFZprimeToMuMu_M_200_dbs0p5_TuneCUETP8M1_13TeV-madgraph-pythia8_16.root'.format(base_path)
outfile = '.'
# set maxEntries = -1 to process all events
maxEntries = 10000
dataYear = '2016'
runPeriod = ''
triggers = ['HLT_Mu50', 'HLT_TkMu50', 'HLT_DoubleEle33_CaloIdL_MW', 'HLT_DoubleEle33_CaloIdL_GsfTrkIdVL']
btagWP = 0.3093 
btag_type = 'deepflavour'
selector = 'bffv2'
keep_and_drop = 'keep_and_drop_bff.txt'
crab = False

In [4]:
#select right module
from PhysicsTools.NanoAODTools.postprocessing.modules.common.heepV72018PromptProducer import heepV72018PromptProducer
if isMC:
    
    from PhysicsTools.NanoAODTools.postprocessing.modules.jme.jetmetHelperRun2 import createJMECorrector
    if dataYear=='2016':
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.lepSFProducer import lepSF2016 as lepSF
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.puWeightProducer import puWeight_2016 as puWeight
        from PhysicsTools.NanoAODTools.postprocessing.modules.btv.btagSFProducer import btagSF2016 as btagSF
    if dataYear=='2017':
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.lepSFProducer import lepSF2017 as lepSF
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.puWeightProducer import puWeight_2017 as puWeight
        from PhysicsTools.NanoAODTools.postprocessing.modules.btv.btagSFProducer import btagSF2017 as btagSF
    if dataYear=='2018':
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.lepSFProducer import lepSF2018 as lepSF
        from PhysicsTools.NanoAODTools.postprocessing.modules.common.puWeightProducer import puWeight_2018 as puWeight
        from PhysicsTools.NanoAODTools.postprocessing.modules.btv.btagSFProducer import btagSF2018 as btagSF
if dataYear=='2016':
    from PhysicsTools.NanoAODTools.postprocessing.modules.common.muonScaleResProducer import muonScaleRes2016 as muonScaleRes
if dataYear=='2017':
    from PhysicsTools.NanoAODTools.postprocessing.modules.common.muonScaleResProducer import muonScaleRes2017 as muonScaleRes
if dataYear=='2018':
    from PhysicsTools.NanoAODTools.postprocessing.modules.common.muonScaleResProducer import muonScaleRes2018 as muonScaleRes

from PhysicsTools.NanoAODTools.postprocessing.modules.bff.WeightsProducer import WeightsProducer
    
#different preselection producerts
if selector=='inclusive':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffInclusive_preselectionModule import bffInclusivePreselProducer as preselectorProducer
if selector=='bff':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffPreselModule import bffPreselProducer as preselectorProducer
if selector=='bffv2':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffPreselModuleV2 import bffPreselProducer as preselectorProducer
if selector=='bff_eff':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffBtagEff import bffBtagEffProducer as preselectorProducer    
if selector=='minseok':
    from PhysicsTools.NanoAODTools.postprocessing.bff.bffPreselModule_minseok import bffPreselProducer as preselectorProducer

#prepare file if glob
if not crab:
    print("using input file", infile)
    import glob
    infile = glob.glob(infile)
else:
    from PhysicsTools.NanoAODTools.postprocessing.framework.crabhelper import inputFiles
    print("get input file (crab)")
    infile = inputFiles()
print("infile", infile)
#2017 EE fix
#set post fix for mc to be T1Smear
metBranchPostFix = 'T1'
if isMC:
    metBranchPostFix = '_T1Smear'
conditions_dict = {
    '2016':{'metBranchName':'MET', 'heepBranchName': 'cutBased_HEEP', 'metBranchPostFix': metBranchPostFix},
    '2017':{'metBranchName':'METFixEE2017', 'heepBranchName': 'cutBased_HEEP', 'metBranchPostFix': metBranchPostFix},
    '2018':{'metBranchName':'MET', 'heepBranchName': 'cutBased_HEEPV7p0_2018Prompt', 'metBranchPostFix': metBranchPostFix}
}




('using input file', '/eos/cms/store/group/phys_exotica/bffZprime/sample_files/BFFZprimeToMuMu_M_200_dbs0p5_TuneCUETP8M1_13TeV-madgraph-pythia8_16.root')
('infile', ['/eos/cms/store/group/phys_exotica/bffZprime/sample_files/BFFZprimeToMuMu_M_200_dbs0p5_TuneCUETP8M1_13TeV-madgraph-pythia8_16.root'])


In [5]:
#set up process for mc and data
if isMC:
    applyHEMfix=True
    jmeCorrections = createJMECorrector(
            isMC=isMC,
            dataYear=dataYear,
            runPeriod=runPeriod,
            jesUncert="Total",
            metBranchName=conditions_dict[dataYear]['metBranchName'],
            applySmearing=True,
            jetType="AK4PFchs",
            noGroom=False,
            applyHEMfix=applyHEMfix,
        )
    
    #don't use all modules for bff efficency calculation
    if 'eff' not in selector:
        modules=[
                #countHistogramsProducer(),
                #triggerFilter(triggers),
                btagSF(),
                jmeCorrections(),
                #puWeight(),
                muonScaleRes(),
                #lepSF(),
                heepV72018PromptProducer(),
                preselectorProducer(btagWP, triggers, isMC=isMC, btag_type=btag_type, record_dataframe=True, applyHEMfix=applyHEMfix, **conditions_dict[dataYear])
        ]
        
        if selector=='bffv2':
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, muon_pt = "correctedUp_pt", **conditions_dict[dataYear]))
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, muon_pt = "correctedDown_pt", **conditions_dict[dataYear]))
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jerDown", **conditions_dict[dataYear]))
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jerUp", **conditions_dict[dataYear]))
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jesTotalDown", **conditions_dict[dataYear]))
            modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jesTotalUp", **conditions_dict[dataYear]))
            if applyHEMfix:
                    modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jesHEMIssueDown", **conditions_dict[dataYear]))
                    modules.append(preselectorProducer(btagWP, triggers, isMC=isMC, applyHEMfix=applyHEMfix, jet_sys = "jesHEMIssueUp", **conditions_dict[dataYear]))
    else:
            modules=[
                btagSF(dataYear, algo=btag_type),
                jmeCorrections(),
                preselectorProducer(btagWP, triggers, isMC=isMC, btag_type=btag_type, record_dataframe=True, **conditions_dict[dataYear])
            ]
    p = PostProcessor(outfile,
            infile,
            modules=modules,
            provenance=True,
            fwkJobReport=True,
            outputbranchsel=keep_and_drop,
            maxEntries=maxEntries
        )
else:
    modules=[
            countHistogramsProducer(),
            triggerFilter(triggers),
            muonScaleRes(),
            heepV72018PromptProducer(),
            preselectorProducer(btagWP, triggers, isMC=isMC, btag_type=btag_type, record_dataframe=True, **conditions_dict[dataYear])
        ]

    p = PostProcessor(outfile,
            infile,
            modules=modules,
            provenance=True,
            fwkJobReport=True,
            outputbranchsel=keep_and_drop,
            jsonInput=runsAndLumis(),
            maxEntries=maxEntries
        )

modules

JEC : Summer16_07Aug2017_V11_MC	 JER : Summer16_25nsV1_MC
MET branch : MET
BTagging: using file DeepJet_2016LegacySF_V1.csv in era 2016 for algo deepflavour
Load Library 'CondFormatsBTauObjects'
Load Library 'CondToolsBTau'
Loading C++ helper from /afs/cern.ch/work/r/rymuelle/public/nanoAODzPrime/CMSSW_10_6_24/src/PhysicsTools/NanoAODTools/python/postprocessing/data/roccor.Run2.v3/RoccoR.cc
Because you requested a FJR we assume you want the final hadd. No name specified for the output file, will use tree.root


In [6]:
p.run()

Will write selected trees to .
Loading jet energy scale (JES) uncertainties from file '/tmp/rymuelle/tmptdYmUl/Summer16_07Aug2017_V11_MC_Uncertainty_AK4PFchs.txt'
Loading jet energy resolutions (JER) from file '/tmp/rymuelle/tmpfi0Rzj/Summer16_25nsV1_MC_PtResolution_AK4PFchs.txt'
Loading JER scale factors and uncertainties from file '/tmp/rymuelle/tmpfi0Rzj/Summer16_25nsV1_MC_SF_AK4PFchs.txt'
Pre-select 10000 entries out of 10000 (100.00%)
nanoAODv5 or higher: True
_jet_nom_muon_corrected_pt_ele_pt
_jet_nom_muon_correctedUp_pt_ele_pt
_jet_nom_muon_correctedDown_pt_ele_pt
_jet_jerDown_muon_corrected_pt_ele_pt
_jet_jerUp_muon_corrected_pt_ele_pt
_jet_jesTotalDown_muon_corrected_pt_ele_pt
_jet_jesTotalUp_muon_corrected_pt_ele_pt
_jet_jesHEMIssueDown_muon_corrected_pt_ele_pt
_jet_jesHEMIssueUp_muon_corrected_pt_ele_pt
Processed 10000 preselected entries from /eos/cms/store/group/phys_exotica/bffZprime/sample_files/BFFZprimeToMuMu_M_200_dbs0p5_TuneCUETP8M1_13TeV-madgraph-pythia8_16.root (10

Error in <TTree::SetBranchStatus>: unknown branch -> DiLepMass
Error in <TTree::SetBranchStatus>: No branch name is matching wildcard -> Inc*
Error in <TTree::SetBranchStatus>: No branch name is matching wildcard -> *puWeight*
Error in <TTree::SetBranchStatus>: unknown branch -> HLT_TkMu100
Error in <TTree::SetBranchStatus>: unknown branch -> HLT_OldMu100
Error in <TTree::SetBranchStatus>: unknown branch -> HLT_DoubleEle25_CaloIdL_MW
Error in <TTree::SetBranchStatus>: unknown branch -> inNregions
Error in <TTree::SetBranchStatus>: No branch name is matching wildcard -> *Btag
Error in <TTree::SetBranchStatus>: unknown branch -> nLep
Error in <TTree::SetBranchStatus>: unknown branch -> nLowPtLep
Error in <TTree::SetBranchStatus>: No branch name is matching wildcard -> Weight*


In [7]:
import uproot as upr

In [13]:
upf = upr.open('BFFZprimeToMuMu_M_200_dbs0p5_TuneCUETP8M1_13TeV-madgraph-pythia8_16_Skim_v1.root')['Events']

In [9]:
upf.keys()

['run',
 'CaloMET_phi',
 'CaloMET_pt',
 'CaloMET_sumEt',
 'ChsMET_phi',
 'ChsMET_pt',
 'ChsMET_sumEt',
 'nCorrT1METJet',
 'CorrT1METJet_area',
 'CorrT1METJet_eta',
 'CorrT1METJet_muonSubtrFactor',
 'CorrT1METJet_phi',
 'CorrT1METJet_rawPt',
 'nElectron',
 'Electron_deltaEtaSC',
 'Electron_dr03EcalRecHitSumEt',
 'Electron_dr03HcalDepth1TowerSumEt',
 'Electron_dr03TkSumPt',
 'Electron_dr03TkSumPtHEEP',
 'Electron_dxy',
 'Electron_dxyErr',
 'Electron_dz',
 'Electron_dzErr',
 'Electron_eCorr',
 'Electron_eInvMinusPInv',
 'Electron_energyErr',
 'Electron_eta',
 'Electron_hoe',
 'Electron_ip3d',
 'Electron_jetPtRelv2',
 'Electron_jetRelIso',
 'Electron_mass',
 'Electron_miniPFRelIso_all',
 'Electron_miniPFRelIso_chg',
 'Electron_mvaFall17V1Iso',
 'Electron_mvaFall17V1noIso',
 'Electron_mvaFall17V2Iso',
 'Electron_mvaFall17V2noIso',
 'Electron_mvaSpring16GP',
 'Electron_mvaSpring16HZZ',
 'Electron_pfRelIso03_all',
 'Electron_pfRelIso03_chg',
 'Electron_phi',
 'Electron_pt',
 'Electron_r9',
 '

In [12]:
upf2 = upr.open('BFFZprimeToMuMu_M_200_dbs0p5_TuneCUETP8M1_13TeV-madgraph-pythia8_16_Skim_v2.root')['Events']

In [15]:
upf2.keys()

['run',
 'CaloMET_phi',
 'CaloMET_pt',
 'CaloMET_sumEt',
 'ChsMET_phi',
 'ChsMET_pt',
 'ChsMET_sumEt',
 'nCorrT1METJet',
 'CorrT1METJet_area',
 'CorrT1METJet_eta',
 'CorrT1METJet_muonSubtrFactor',
 'CorrT1METJet_phi',
 'CorrT1METJet_rawPt',
 'nElectron',
 'Electron_deltaEtaSC',
 'Electron_dr03EcalRecHitSumEt',
 'Electron_dr03HcalDepth1TowerSumEt',
 'Electron_dr03TkSumPt',
 'Electron_dr03TkSumPtHEEP',
 'Electron_dxy',
 'Electron_dxyErr',
 'Electron_dz',
 'Electron_dzErr',
 'Electron_eCorr',
 'Electron_eInvMinusPInv',
 'Electron_energyErr',
 'Electron_eta',
 'Electron_hoe',
 'Electron_ip3d',
 'Electron_jetPtRelv2',
 'Electron_jetRelIso',
 'Electron_mass',
 'Electron_miniPFRelIso_all',
 'Electron_miniPFRelIso_chg',
 'Electron_mvaFall17V1Iso',
 'Electron_mvaFall17V1noIso',
 'Electron_mvaFall17V2Iso',
 'Electron_mvaFall17V2noIso',
 'Electron_mvaSpring16GP',
 'Electron_mvaSpring16HZZ',
 'Electron_pfRelIso03_all',
 'Electron_pfRelIso03_chg',
 'Electron_phi',
 'Electron_pt',
 'Electron_r9',
 '

In [21]:
upf2.arrays(['SR1_jet_nom_muon_corrected_pt_ele_pt'], library='np')['SR1_jet_nom_muon_corrected_pt_ele_pt'].sum()

860

In [24]:
upf.arrays(['SR1_nom'], library='np')['SR1_nom'].sum()

847

In [ ]:
upf.arrays(['SR1_nom'], library='np')['SR1_nom'].sum()

In [25]:
upf2.arrays(['SR1_jet_jesHEMIssueUp_muon_corrected_pt_ele_pt'], library='np')['SR1_jet_jesHEMIssueUp_muon_corrected_pt_ele_pt'].sum()

43

In [ ]:
df = upf.arrays(['Muon_corrected_pt',
 'Muon_correctedUp_pt',
 'Muon_correctedDown_pt',
 'Muon_effSF_trigger',
 'Muon_effSF_stat_trigger',
 'Muon_effSF_ID',
 'Muon_effSF_stat_ID',
 'Muon_effSF_ISO',
 'Muon_effSF_stat_ISO',
 'Muon_effSF_sys_ID',
 'Muon_effSF_sys_ISO'], library='pd')

In [ ]:
df.Muon_effSF_trigger*df.Muon_effSF_ID*df.Muon_effSF_ISO

In [ ]:
trig = (df.Muon_effSF_trigger+df.Muon_effSF_stat_trigger)

In [ ]:
id = (df.Muon_effSF_ID+(df.Muon_effSF_sys_ID**2+df.Muon_effSF_sys_ID**2)**.5)

In [ ]:
iso = (df.Muon_effSF_ISO+(df.Muon_effSF_sys_ISO**2+df.Muon_effSF_stat_ISO**2)**.5)

In [ ]:
(trig*id*iso).mean()

In [ ]:
df = upf.arrays(['Jet_pt_jesHEMIssueDown', 'Jet_pt_jesTotalDown', 'Jet_pt_nom', 'RelMET_nom', 'RelMET_jesHEMIssueDown' ,'SR1_nom', 'SR1_jesHEMIssueDown'], library='pd')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df[(df.SR1_nom!=df.SR1_jesHEMIssueDown) ]

In [ ]:
def get_keys(key):
    return [x for x in upf.keys() if key in x]

In [ ]:
df = upf.arrays(['HTLT_nom', 'HTLT_jesHEMIssueDown', 'SR1_nom', 'SR1_jesHEMIssueDown', 'SR1_jesTotalDown', 'Jet_pt_jesTotalDown', 'Jet_pt_jesHEMIssueDown', 'Jet_pt_nom'], library='pd')

In [ ]:
df[(df.SR1_nom!=df.SR1_jesTotalDown)]

In [ ]:
get_keys('HEM')